In [1]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players.xlsx', index_col=0)


data_folder = '../Data/'
file_names = ['Serie_A']


In [2]:
years = range(2013, 2014)


for year in years:
    if year == 2016:
        pass
    else:
        print(year)
        df_jogos_2014 = pd.read_json(f'../Data/Serie_A_{year}_games.json')
        df_jogos_2014_ = df_jogos_2014.T


        for index, row in df_jogos_2014_.iterrows():
            processor = FootballDataProcessor(df_jogos_2014_, index-1)
            final_df = processor.process(data_folder, file_names, year)
            final_df.rename(columns={'player_id': 'player'}, inplace=True)

            final_df['player'] = final_df['player'].astype('string')
            id_players['player'] = id_players['player'].astype('string')

            final_df_ = pd.merge(final_df, id_players, on='player', how='left')

            team_1_name = df_jogos_2014_.iloc[index-1,0]
            team_2_name = df_jogos_2014_.iloc[index-1,1]


            # Criar uma instância de FootballMatchRating
            rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

            # Atualizar as ratings
            updated_match_data = rating_calculator.update_ratings()
            # Garantir que 'player' em updated_match_data está como string
            updated_match_data['player'] = updated_match_data['player'].astype('string')
            # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
            updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
            updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

            updated_values = id_players[['player']].merge(
                updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
                on='player',
                how='left'
            )

            # Atualizar apenas as linhas corespondentes em id_players
            id_players.update(updated_values)

id_players


2013


,player,rating,age,games_played,player_name,time_jogador
0,429638,1500.000000,25,50,NaN,NaN
1,563757,1500.000000,25,50,NaN,NaN
2,643996,1500.000000,25,50,NaN,NaN
3,816320,1500.000000,25,50,NaN,NaN
4,372444,1500.000000,25,50,NaN,NaN
...,...,...,...,...,...,...
2881,777158,1500.000000,25,50,NaN,NaN
2882,356106,1500.000000,25,50,NaN,NaN
2883,162466,1500.000000,25,50,NaN,NaN
2884,541294,1500.000000,25,50,NaN,NaN


In [3]:
x = id_players.sort_values(by='rating', ascending=False)
x.to_excel('teste.xlsx')
x

,player,rating,age,games_played,player_name,time_jogador
2642,159747,1529.076670,25,74,9Humberlito Humberlito Borges,Cruzeiro / MG
2492,387195,1527.448394,25,57,20Martinucio Alejandro Hernan Mar...,Cruzeiro / MG
2293,348895,1525.109518,25,60,19Biteco Guilherme Bitencourt...,Grêmio / RS
1993,177749,1519.584569,25,81,26Dede Anderson Vital da Silva,Cruzeiro / MG
682,170812,1517.804562,25,59,8Henrique Henrique Pacheco de Lima,Cruzeiro / MG
...,...,...,...,...,...,...
2790,298595,1226.652754,25,84,27Luan Luan Madson Gedeao d ...,Atlético Mineiro / MG
2229,176512,1226.258588,25,76,10Tiago Real,Náutico / PE
344,166491,1219.367739,25,79,8Derley Wanderley de Jesus Sousa,Náutico / PE
1837,177998,1188.883915,25,74,6Bruno Bruno Bairros Collaco,Náutico / PE
